In [7]:
import base64
import pandas as pd
import io
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
from dash.dependencies import State

In [8]:
# Load CSV with all the H-Pile info
H_Pile = pd.read_csv("https://raw.githubusercontent.com/geotechnick/Geotechnical_Design/main/Pile_Design/H_Piles/HP_Spec_Table.csv")
Fy = 50  # Pile yield stress in ksi
pile_names = H_Pile['Section_E'].unique().tolist()

# Filter out the metric values
H_Pile = H_Pile.loc[:, ~H_Pile.columns.str.endswith('_M')]

# Create drop down with the pile names
dropdown = dcc.Dropdown(id='pile-dropdown', options=[{'label': name, 'value': name} for name in pile_names], value=pile_names[0])

# Define app layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Pile Information"),
    dropdown,
    html.Div(id='pile-info'),

    html.H1("File Upload"),
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=False
    ),
    html.Div(id='output-data-upload'),
])

# Callback to display pile info
@app.callback(
    Output('pile-info', 'children'),
    [Input('pile-dropdown', 'value')]
)
def display_row(selected_value):
    if selected_value is None:
        return html.Div("No pile selected")
    
    pile_info = H_Pile[H_Pile['Section_E'] == selected_value]
    pile_units = H_Pile[H_Pile['Section_E'] == 'Units_English']
    pile_display = pd.concat([pile_info, pile_units], ignore_index=True)
    
    # Adds case_1 to the data frame
    old_index_label = pile_info.index[0]
    new_index_label = 'case_1'
    pile_info = pile_info.rename(index={old_index_label: new_index_label})
    pile_info.reset_index(inplace=True)

    return html.Table([html.Tr([html.Th(col), html.Td(pile_display[col].values[0])]) for col in pile_display.columns])

# Function to handle file upload
def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = io.StringIO(base64.b64decode(content_string).decode('utf-8'))

    df = pd.read_csv(decoded)
    
    # Create Plotly table
    table = go.Figure(data=[go.Table(
        header=dict(values=df.columns,
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[df[column] for column in df.columns],
                   fill_color='lavender',
                   align='left'))
    ])
    
    return html.Div([
        html.H5(filename),
        html.H6("Raw Data:"),
        html.Pre(df.to_string()),
        html.H6("Interactive Table:"),
        dcc.Graph(
            id='table',
            figure=table
        ),
    ])


@app.callback(Output('output-data-upload', 'children'),
              [Input('upload-data', 'contents')],
              [State('upload-data', 'filename')])
def update_output(contents, filename):
    if contents is not None:
        children = parse_contents(contents, filename)
        return children

if __name__ == '__main__':
    app.run_server(debug=True)

In [4]:
# Define app layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("File Upload"),
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-data-upload')
])

# Function to handle file upload
def parse_contents(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    
    try:
        if 'csv' in filename:
            # Assume the uploaded file is a CSV file
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume the uploaded file is a Excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.Hr(),  # horizontal line
        # Display the first 10 rows of the uploaded file
        html.Div(df.head(10))
    ])

# Callback to update output based on file upload
@app.callback(Output('output-data-upload', 'children'),
              [Input('upload-data', 'contents')],
              [State('upload-data', 'filename')])
def update_output(list_of_contents, list_of_names):
    if list_of_contents is None or list_of_names is None:
        raise PreventUpdate
    
    children = [
        parse_contents(c, n) for c, n in
        zip(list_of_contents, list_of_names)]
    return children

if __name__ == '__main__':
    app.run_server(debug=True)